1. 爬wiki

In [1]:
import httpx
from bs4 import BeautifulSoup
import time

url = "https://wikiwiki.jp/wds/%E5%8F%8E%E9%8C%B2%E6%A5%BD%E6%9B%B2%E4%B8%80%E8%A6%A7"

response = httpx.get(url, verify=False)

if response.status_code != 200:
    raise Exception(f"Failed to fetch page: {response.status_code}")

soup = BeautifulSoup(response.content, "html.parser")
rows = soup.findAll("div", class_="h-scrollable")[3].contents[0].contents[1].contents
time.sleep(10)
infos = []
for row in rows:
    """
    wiki id
    wiki status
    曲名 *
    演出队名 *
    1-5难度定数
    4-5难度combo数
    时长
    体力
    BPM
    ？
    上线日期 *
    曲绘（小）
    演唱者
    wiki编辑
    """
    detail_url = "https://wikiwiki.jp" + row.contents[2].contents[0]["href"]
    print(detail_url)
    # print(detail_url)
    response = httpx.get(detail_url, verify=False)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch page: {response.status_code}")
    # 每首歌曲的wiki页面
    soup_detail = BeautifulSoup(response.content, "html.parser")
    response = httpx.get(detail_url, verify=False)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch page: {response.status_code}")

    soup_detail = BeautifulSoup(response.content, "html.parser")
    # soup_detail
    detail_rows = (
        soup_detail.findAll("div", class_="h-scrollable")[0]
        .contents[0]
        .contents[0]
        .contents
    )
    """
        曲绘（大）
        \
        作曲 ， 作词
        \
        编曲 ， 歌唱（列表）
        BPM ， 演奏时间 ， 体力消费
        \
        \ N H E S O（定数）
        \ N H E S O（notes）
        
    """
    info = {}
    info["image"] = (
        detail_rows[0].contents[0].contents[0]["href"]
        if len(detail_rows[0].contents[0].contents) == 1
        else detail_rows[0].contents[0].contents[1]["href"]
    )  # .split(r'?')[0]
    info["artist_composition"] = detail_rows[2].contents[0].text
    info["artist_lyrics"] = detail_rows[2].contents[1].text
    info["artist_arrangement"] = detail_rows[4].contents[0].text
    info["artist_vocals"] = detail_rows[4].contents[1].text
    info["bpm"] = detail_rows[5].contents[1].text
    info["legnth"] = detail_rows[5].contents[3].text
    info["cost"] = detail_rows[5].contents[5].text
    info["level"] = ["Normal", "Hard", "Extra", "Stella", "Olivier"]
    info["base"] = [a.text for a in detail_rows[7].contents[1:6]]
    info["combo"] = [a.text for a in detail_rows[8].contents[1:6]]
    # info
    info['title'] = row.contents[2].text
    info['team'] = row.contents[3].text
    info['release'] = row.contents[15].text
    infos.append(info)


<>:55: SyntaxWarning: invalid escape sequence '\ '
<>:55: SyntaxWarning: invalid escape sequence '\ '
C:\Users\sofee\AppData\Local\Temp\ipykernel_18340\1449894529.py:55: SyntaxWarning: invalid escape sequence '\ '
  """


https://wikiwiki.jp/wds/%E3%83%AF%E3%83%8A%E3%83%93%E3%82%B9%E3%82%BF%EF%BC%81
https://wikiwiki.jp/wds/%E3%83%88%E3%82%A5%E3%83%BB%E3%82%AA%E3%83%96%E3%83%BB%E3%82%A2%E3%82%B9
https://wikiwiki.jp/wds/%E5%8B%BF%E5%BF%98%E5%94%84
https://wikiwiki.jp/wds/%E5%A4%A2%E8%A6%8B%E6%9C%88%E5%A4%9C
https://wikiwiki.jp/wds/%E6%82%A0%E4%B9%85%E6%9C%88%E8%8F%AF
https://wikiwiki.jp/wds/%E3%83%80%E3%82%A4%E3%83%A4%E3%83%A2%E3%83%B3%E3%83%89%E3%81%AE%E8%AA%93%E3%81%84
https://wikiwiki.jp/wds/%E9%AD%94%E6%B3%95%E3%81%AE%E3%83%A9%E3%82%B9%E3%83%88%E3%83%8E%E3%83%BC%E3%83%88
https://wikiwiki.jp/wds/New%20Nostalgic%20Friend
https://wikiwiki.jp/wds/Drawing%20World
https://wikiwiki.jp/wds/%E5%90%8D%E3%82%82%E3%81%AA%E3%81%8D%E6%81%8B%E4%BA%BA%E3%82%88
https://wikiwiki.jp/wds/Etoile
https://wikiwiki.jp/wds/Masquerade
https://wikiwiki.jp/wds/Farewell%20song
https://wikiwiki.jp/wds/%E3%82%B7%E3%83%AA%E3%82%A6%E3%82%B9%E3%81%AE%E8%BC%9D%E3%81%8D%E3%81%AE%E3%82%88%E3%81%86%E3%81%AB
https://wikiwiki.jp/wds/Faith%2

2. 爬解包站

In [2]:
import re
import json

url = "https://redive.estertion.win/wds/jacket/"

response = httpx.get(url, verify=False)

if response.status_code != 200:
    raise Exception(f"Failed to fetch page: {response.status_code}")

names = json.loads(re.search(r'names=([^\n\r]*)', response.content.decode('utf-8')).group(1).strip()[0:-1])

In [3]:
name2id = {}
for cid in names:
    name2id[names[cid]]=eval(cid)


In [7]:
import difflib
for info in infos:
    info['id'] = name2id[difflib.get_close_matches(info['title'], list(name2id.keys()), n=1)[0]]

In [9]:
import json

with open("wds.json", "w", encoding='utf-8') as f:
    json.dump(infos, f, ensure_ascii=False, indent=4)

谱面文件：GET https://redive.estertion.win/wds/Notations/{谱面ID}/{难度（1-5）}.txt
谱面图像：GET https://redive.estertion.win/wds/Notations/{谱面ID}/{难度（1-5）}.svg